In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# load data
df=pd.read_csv('HR_comma_sep.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
sales                    14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
print(df.left.value_counts())

x=round(df.left.value_counts()/len(df)*100,2)
print(x[1],'% of employees left')
print(x[0],'% of employees stayed')


0    11428
1     3571
Name: left, dtype: int64
23.81 % of employees left
76.19 % of employees stayed


In [4]:
all_Y=df[['left']]
del df['left']
all_X=df

In [6]:
all_Y[['left']].drop_duplicates()

,left
0,1
2000,0


In [7]:
all_X[['sales']].drop_duplicates()

,sales
0,sales
28,accounting
31,hr
35,technical
46,support
60,management
61,IT
66,product_mng
77,marketing
301,RandD


In [8]:
    enc = LabelEncoder()
    sales_feture=enc.fit_transform(all_X[['sales']])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
def get_onehot_df(data_array):
    enc = LabelEncoder()
    sales_feture=enc.fit_transform(data_array)
    oneenc = OneHotEncoder()
    sales_hotfeture=oneenc.fit_transform(sales_feture.reshape(-1,1)).toarray()
    return sales_hotfeture

In [17]:
sales_hotfeture=get_onehot_df(all_X[['sales']])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
sales_hotfeture.shape

(14999, 10)

In [20]:
X_data=np.concatenate((np.array(all_X.iloc[0:,0:-2]),sales_hotfeture),axis=1)

In [21]:
X_data.shape

(14999, 17)

In [22]:
all_X[['salary']].drop_duplicates()

,salary
0,low
1,medium
72,high


In [23]:
salary_hotfeture=get_onehot_df(all_X[['salary']])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
X_data=np.concatenate((X_data,salary_hotfeture),axis=1)

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X_data,all_Y,test_size=0.3)

In [32]:
clf=tree.DecisionTreeClassifier()

In [33]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [34]:
y_pred=clf.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(y_pred,y_test)

0.97799999999999998